In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df=pd.read_csv("Product.csv")

In [5]:
df.head()

,title,Category,selling_price,product_rating,user_id,links
0,Alisha Solid Women's Cycling Shorts,"""Clothing Women's Clothing Lingerie, Sleep &...",999,5.0,19,https://m.media-amazon.com/images/I/51MmfNZb6T...
1,FabHomeDecor Fabric Double Sofa Bed,"""Furniture Living Room Furniture Sofa Beds &...",32157,3.0,131,/static/img/Image_not_available.png\n
2,AW Bellies,"""Footwear Women's Footwear Ballerinas AW Be...",999,1.6,166,https://m.media-amazon.com/images/I/61DlDX5L0e...
3,Alisha Solid Women's Cycling Shorts,"""Clothing Women's Clothing Lingerie, Sleep &...",699,3.4,12,https://m.media-amazon.com/images/I/51MmfNZb6T...
4,Sicons All Purpose Arnica Dog Shampoo,"""Pet Supplies Grooming Skin & Coat Care Sha...",220,2.9,137,https://m.media-amazon.com/images/I/61OCfq7o8F...


In [6]:
df.describe()

,product_rating,user_id
count,31952.000000,31952.000000
mean,3.440589,99.534646
std,1.075193,57.899824
min,1.000000,0.000000
25%,2.700000,49.000000
50%,3.800000,99.000000
75%,4.200000,149.000000
max,5.000000,200.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31952 entries, 0 to 31951
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           31952 non-null  object 
 1   Category        31952 non-null  object 
 2   selling_price   31874 non-null  object 
 3   product_rating  31952 non-null  float64
 4   user_id         31952 non-null  int64  
 5   links           31952 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 1.5+ MB


In [8]:
df.isna().sum()

title              0
Category           0
selling_price     78
product_rating     0
user_id            0
links              0
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.duplicated().sum()

6

In [11]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(23045, 6)

In [17]:
df.drop_duplicates('title',inplace=True)

In [18]:
tfv = TfidfVectorizer(min_df=3,strip_accents='unicode',analyzer='word',
token_pattern=r'\w{1,}',ngram_range=(1,3),stop_words="english")

In [19]:
tfv_matrix=tfv.fit_transform(df['title'])

In [20]:
sig= sigmoid_kernel(tfv_matrix,tfv_matrix)

In [21]:
sig[0]

array([0.76160949, 0.76159416, 0.76159416, ..., 0.76159546, 0.76159416,
       0.76159436])

In [22]:
indexs=pd.Series(df.index,index=df["title"]).drop_duplicates()

In [23]:
indexs

title
Alisha Solid Women's Cycling Shorts                                                                             0
FabHomeDecor Fabric Double Sofa Bed                                                                             1
AW Bellies                                                                                                      2
Sicons All Purpose Arnica Dog Shampoo                                                                           4
Eternal Gandhi Super Series Crystal Paper Weights  with Silver Finish                                           5
                                                                                                            ...  
Whiskas (+1 year) Ocean Fish 3 kg Dry Adult Cat Food                                                        31947
BASE 41 Women Solid Round Neck Red T-Shirt                                                                  31948
HRX by Hrithik Roshan Solid Women Black Tights                                    

In [24]:
sig[50]

array([0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [25]:
indexs=indexs.reset_index()

In [26]:
indexs[indexs['title']=="ITWOSERVICES CRICKET NET 100X10 CRICKET NET NYLON HDPE Cricket Net  (Green)"].index[0]

13927

In [27]:
def giverecomm(title,sig=sig):
  idx=indexs[indexs['title']==title].index[0]
  sig_scores=list(enumerate(sig[idx]))
  sig_scores=sorted(sig_scores,key= lambda x:x[1],reverse=True)
  sig_scores=sig_scores[1:11]

  product_index=[i[0] for i in sig_scores]

  return df['title'].iloc[product_index]

In [28]:
m=df['title'][1000]
print(m)
giverecomm(m)

INDRICKA Casual Roll-up Sleeve Solid Women's Top


1023        INDRICKA Casual Full Sleeve Solid Women's Top
3187    PURYS Casual Roll-up Sleeve Printed Women's Wh...
9641    Ir Acc Casual Roll-up Sleeve Self Design Women...
989     INDRICKA Casual Butterfly Sleeve Solid Women's...
656      Imagination Casual Full Sleeve Solid Women's Top
661           Besiva Casual Full Sleeve Solid Women's Top
863           Schwof Casual Full Sleeve Solid Women's Top
2946    I Am For You Casual Full Sleeve Solid Women's Top
9609         De Moza Casual Full Sleeve Solid Women's Top
9809            Bebo Casual Full Sleeve Solid Women's Top
Name: title, dtype: object

## Saving Files

In [29]:
joblib.dump(df,"content.sav")

['content.sav']

In [30]:
for i in range(len(sig)):
    np.save(f"Sigmoid/sig{i}.npy",sig[i])

In [40]:
joblib.dump(indexs,"Indexs.sav")

['Indexs.sav']